# PUBG 最终排名预测

## 线性回归预测模型

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import os

### 1. 读取数据集

In [2]:
pubg_train=pd.read_csv( '/home/mqli/data-mining/final-work/train_V2.csv')
pubg_train

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.000,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.040,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.000,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.000,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.000,0,0,49.75,2,0,0.1875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446961,afff7f652dbc10,d238e426f50de7,18492834ce5635,0,0,0.00,0,0,0,74,...,0,1292.0000,0,0.000,0,0,1019.00,3,1507,0.1786
4446962,f4197cf374e6c0,408cdb5c46b2ac,ee854b837376d9,0,1,44.15,0,0,0,69,...,0,0.0000,0,0.000,0,0,81.70,6,0,0.2935
4446963,e1948b1295c88a,e26ac84bdf7cef,6d0cd12784f1ab,0,0,59.06,0,0,0,66,...,0,0.0000,0,2.184,0,0,788.70,4,0,0.4815
4446964,cc032cdd73b7ac,c2223f35411394,c9c701d0ad758a,0,4,180.40,1,1,2,11,...,2,0.0000,0,0.000,0,0,2748.00,8,0,0.8000


### 2. 缺失值处理

In [3]:
#将所有缺失值填充为0
pubg_train=pubg_train.fillna(0)
#将matchType列数据转化为适合模型的数据，相当于一个标准化处理
Le=LabelEncoder()
pubg_train['matchType']=Le.fit_transform(pubg_train['matchType'])

### 3. 特征选择与模型训练

In [4]:
from sklearn.model_selection import train_test_split
X_train=pubg_train.drop(['winPlacePerc','Id','matchId','groupId'],axis=1)
Y_train=np.array(pubg_train['winPlacePerc'])
random_seed=1
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)
#创建线性回归模型
model=LinearRegression()
#训练线性回归模型
model=model.fit(X_train,Y_train)
#用线性回归模型进行预测
pred=model.predict(X_train)

### 4. 模型评价

In [5]:
from sklearn.metrics import mean_absolute_error
print('MAE of train: ', mean_absolute_error(pred, Y_train))
print('MAE of val: ', mean_absolute_error(model.predict(X_val), Y_val))

MAE of train:  0.09199775645256865
MAE of val:  0.09222107817896855


### 5. 保存测试集预测排名信息

In [6]:
pred_val = model.predict(X_val)
test_df = X_val
test_df['winPlacePerc'] = Y_val
test_df['pred_winPlacePerc'] = pred_val
test_df.to_csv('/home/mqli/data-mining/final-work/test_pred_lr.csv')